In [7]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=6

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],0:2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=3
p=floor(Int64,n/u)

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
f=sum(generate_random_poly(I[j]) for j in 1:p)


g=-(x.^2).+1/u



m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)


randx=2*rand(n).-1
randx=randx./sqrt(u)

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)
println("====================")

k=2

println("Relaxed order: k=",k)
println("====================")

***Problem setting***
Number of variable: n=6
Number of inequality constraints: m=6
Number of equality constraints: l=1
Relaxed order: k=2


In [8]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

In [9]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_CS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
    maxit=Int64(1e10),tol=1e-2,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true)

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
  Number of cliques: p=2
  Largest clique size: u=4


  Largest constant trace: a=4.111111111111111
  Number of blocks: omega=9
  Number of equality consraints: zeta=181
  Size of the largest block: s^max=15
Modeling time:
  0.349861 seconds (214.51 k allocations: 11.990 MiB)
iter=1   val=-4.913628258960475   gap=0.06550414757684797   feas=5.87102908550129
iter=2   val=-3.18665776377629   gap=0.41780328713545445   feas=3.5343297508086575
iter=4   val=-1.9614403328095507   gap=0.35506782354385624   feas=1.6108672293798914
iter=8   val=-2.1362426742334373   gap=0.02930004900371052   feas=0.9359602421260776
iter=16   val=-1.7512048205552633   gap=0.11535604051361374   feas=0.5329382271779893
iter=32   val=-1.8647265163206719   gap=0.0528822446213626   feas=0.19798902825446374
iter=64   val=-1.8929807250565684   gap=0.0115754867290072   feas=0.08785172034198699
iter=128   val=-1.8731517565795417   gap=0.022221776588311457   feas=0.019667757891193443
iter=256   val=-1.8717997116783887   gap=0.016513089080277186   feas=0.013501434555425794
iter

-1.8678673609662508

In [10]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_CS_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
    tol=1e-2,use_eqcons_to_get_constant_trace=false)

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
  Number of cliques: p=2
  Largest clique size: u=4


  Largest constant trace: a=4.111111111111111
  Number of blocks: omega=9
  Number of equality consraints: zeta=181
  Size of the largest block: s^max=15
Modeling time:
  0.396463 seconds (214.52 k allocations: 11.994 MiB)
**LMBM solver:
---------------
| Parameters: |
---------------
n:       181
maxtime: 300000.000000
na:      2
mcu:     5
mc:      7
rpar: 
ipar: 
 Entry to LMBM:
 NIT=    1  NFE=    1  F= 0.76684091D+00  WK= 0.1589D-01  QK= 0.7947D-02
 NIT=    2  NFE=   46  F= 0.76684091D+00  WK= 0.1589D-01  QK= 0.7947D-02
 NIT=    3  NFE=   91  F= 0.76684091D+00  WK= 0.1589D-01  QK= 0.7947D-02
 NIT=    4  NFE=  136  F= 0.76684091D+00  WK= 0.1589D-01  QK= 0.7947D-02
 NIT=    5  NFE=  181  F= 0.76684091D+00  WK= 0.1589D-01  QK= 0.7947D-02
 NIT=    6  NFE=  226  F= 0.76684091D+00  WK= 0.1589D-01  QK= 0.7947D-02
 NIT=    7  NFE=  271  F= 0.76684091D+00  WK= 0.1589D-01  QK= 0.7947D-02
 NIT=    8  NFE=  316  F= 0.76684091D+00  WK= 0.1589D-01  QK= 0.7947D-02
 NIT=    9  NFE=  361  F= 0.766

-4.952004134548726

In [11]:
using TSSOS


@time opt,sol,data=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS=false,CTP=false);

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 100             
  Cones                  : 0               
  Scalar variables       : 16              
  Matrix variables       : 8               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 

In [12]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       20
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       31

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

-1.8714841877676296